In [ ]:
pip install chess

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for chess: filename=chess-1.11.1-py3-none-any.whl size=148497 sha256=d5725f41e64b73e985e9087764faed34b684428ec426f62ab93ee26160bec348
  Stored in directory: /root/.cache/pip/wheels/f0/3f/76/8783033e8524d407e1bebaf72fdd3f3eba27e0c030e92bbd87
Successfully built chess


In [ ]:
!pip install tensorflow==2.14.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.9/489.9 MB 790.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 80.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 66.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.7/440.7 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 6.2 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.17.0
    Uninstalling wrapt-1.17.0:
      Successfully uninstalled wrapt-1.17.0
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: keras
    Found existing installation: keras 3.5.0
    Uninstalling keras-3.5.0:
      Successfully uninstalled keras-3.5.0
  Attempting uninstall: google-auth-oauthlib
    Found existing i

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import chess
import tensorflow as tf
from tensorflow.keras import layers, models
import os
from google.colab import drive
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [ ]:
#pip show tensorflow

Name: tensorflow
Version: 2.14.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /usr/local/lib/python3.11/dist-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, setuptools, six, tensorboard, tensorflow-estimator, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: dopamine_rl, tensorflow-text, tf_keras


In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models

def create_residual_block(input_tensor, filters):
  """"
  Crea los 'residual blocks' de la red
  """
  x = layers.Conv2D(filters, (3, 3), padding="same", activation="relu")(input_tensor)
  x = layers.BatchNormalization()(x)
  x = layers.Conv2D(filters, (3, 3), padding="same", activation="relu")(x)
  x = layers.BatchNormalization()(x)
  x = layers.Add()([input_tensor, x])
  return x

def create_chess_network():
    input_layer = layers.Input(shape=(8, 8, 12))

    x = layers.Conv2D(256, (3, 3), padding="same", activation="relu")(input_layer)
    x = layers.BatchNormalization()(x)

    # Residual blocks
    for _ in range(10): # Adding 10 residual layers
        x = create_residual_block(x, 256)

    x = layers.Flatten()(x)

    x = layers.Dense(1024, activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)

    x = layers.Dense(512, activation="relu")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)

    # Policy output
    policy_head = layers.Dense(4672, activation="softmax", name="policy_output")(x)
    # Value output
    value_head = layers.Dense(1, activation="tanh", name="value_output")(x)

    model = models.Model(inputs=input_layer, outputs=[policy_head, value_head])

    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=0.001,
        decay_steps=1000,
        decay_rate=0.9,
    )
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

    model.compile(
      optimizer=optimizer,
      loss={'policy_output': 'categorical_crossentropy', 'value_output': 'mean_squared_error'}
    )

    return model

In [4]:
import tensorflow as tf
from tensorflow.keras.utils import plot_model

In [5]:
model = create_chess_network()
plot_model(model, to_file='neural_network.png', show_shapes=True, show_layer_names=True)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
class Node:
    def __init__(self, state, parent=None, prior_prob=1.0):
        self.state = state
        self.parent = parent
        self.children = {}
        self.visits = 0
        self.value_sum = 0
        self.prior_prob = prior_prob

    def expand(self, legal_moves, policy_probs):
        """
        Expande el nodo con los movimientos legales y probabilidades de política.
        """
        for move, prob in zip(legal_moves, policy_probs):
            if move not in self.children:
                next_state = self.state.copy()  # Crear un nuevo estado a partir del movimiento
                next_state.push(move)
                self.children[move] = Node(next_state, parent=self, prior_prob=prob)

    def value(self):
        """
        Calcula el valor actual del nodo.
        """
        return self.value_sum / max(1, self.visits)

    def select_child(self, c_puct=0.7):
     """
     Selecciona el mejor hijo usando PUCT.
     """
     selected_child = max(
         self.children.items(),
         key=lambda item: item[1].value() + c_puct * item[1].prior_prob * (self.visits ** 0.5) / (1 + item[1].visits)
     )
     #print (f"selected move: {selected_child[0]}, visits: {selected_child[1].visits}")
     return selected_child

def predict_policy_and_value(network, board):
    """
    Convierte el tablero a tensor y utiliza la red neuronal para predecir política y valor.
    """
    tensor_input = board_to_tensor(board)
    policy, value = network.predict(tensor_input[None, :, :, :])
    return policy[0], value[0][0]

def mcts(root, network, iterations=20, dirichlet_alpha=0.1):
    for _ in range(iterations):
        node = root
        while node.children:
          _, node = node.select_child()

        if not node.state.is_game_over():
            legal_moves = list(node.state.legal_moves)
            policy_probs, value = predict_policy_and_value(network, node.state)

            legal_move_probs = np.zeros(len(legal_moves))
            for i, move in enumerate(legal_moves):
                move_index = move_to_index(move)
                legal_move_probs[i] = policy_probs[move_index]
            legal_move_probs /= np.sum(legal_move_probs) + 1e-8  # Normalizar y evitar división por cero

          # Añade 'Dirichlet noise' al root node para asegurar que se juegan todos los movimientos
            if node == root:
                noise = np.random.dirichlet([dirichlet_alpha] * len(legal_moves))
                legal_move_indexes = [move_to_index(move) for move in legal_moves]
                policy_probs[legal_move_indexes] = policy_probs[legal_move_indexes] + 0.25 * noise

                node.expand(legal_moves, policy_probs)
        else:
            value = {"1-0": 1, "0-1": -1, "1/2-1/2": 0}[node.state.result()]

      # Retropropagación del valor
        while node:
            node.visits += 1
            node.value_sum += value if node.parent else -value  # Propagación opuesta para minimizar/maximizar
            node = node.parent

def select_move_with_temperature(node, tau=1.0):
    if tau == 0:
        return max(node.children.items(), key=lambda item: item[1].visits)[0]

    visits = np.array([child.visits for child in node.children.values()])
    probabilities = np.exp(visits / tau) / np.sum(np.exp(visits / tau))
    moves = list(node.children.keys())
    return np.random.choice(moves, p=probabilities)

def train_network(network, boards, policies, values, epochs=5, batch_size=32, output_dir=None):
    """
    Entrena la red neuronal
    """

    callbacks = [
        EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True,
            verbose=1
        ),
    ]
    if output_dir:
        checkpoint_path = os.path.join(output_dir, "best_model")
        callbacks.append(
            ModelCheckpoint(
                filepath=checkpoint_path,
                monitor='val_loss',
                save_best_only=True,    # Guarda solo los mejores pesos
                save_weights_only=False, # Guarda todo el modelo, no solo los pesos
                verbose=1
            )
        )

    history = network.fit(
        boards,
        {'policy_output': policies, 'value_output': values},
        epochs=epochs,
        batch_size=batch_size,
        validation_split=0.1,
        callbacks=callbacks
    )
    return history

def board_to_tensor(board):
    """
    Convierte un tablero de ajedrez (chess.Board) a una representación tensorial 8x8x12.
    """
    tensor = np.zeros((8, 8, 12), dtype=int)
    piece_map = {
        chess.PAWN: 0, chess.KNIGHT: 1, chess.BISHOP: 2,
        chess.ROOK: 3, chess.QUEEN: 4, chess.KING: 5
    }

    for square, piece in board.piece_map().items():
        row, col = divmod(square, 8)
        channel = piece_map[piece.piece_type]
        if piece.color == chess.BLACK:
            channel += 6  # Canales 6-11 para piezas negras
        tensor[row, col, channel] = 1

    return tensor

def move_to_index(move):
    """
    Convierte un movimiento de ajedrez en notación UCI a un índice entero.
    Utiliza una codificación de posiciones desde (0, 0) hasta (7, 7) y 73 tipos de movimientos.
    """
    uci_move = move.uci()  # Convertir a notación UCI
    from_square = chess.SQUARE_NAMES.index(uci_move[:2])  # Índice de casilla de origen
    to_square = chess.SQUARE_NAMES.index(uci_move[2:4])  # Índice de casilla de destino

    if len(uci_move) == 5:  # Movimientos con promoción
        promotion_piece = uci_move[4]
        promotion_index = {'q': 0, 'r': 1, 'b': 2, 'n': 3}[promotion_piece]
        return from_square * 64 + to_square * 4 + promotion_index
    else:
        return from_square * 64 + to_square

def index_to_move(index, board):
    """
    Convierte un índice a un movimiento de ajedrez usando el estado del tablero.
    """
    from_square = index // 64
    to_square = (index % 64) // 4
    promotion_piece = index % 4
    promotion_dict = {0: 'q', 1: 'r', 2: 'b', 3: 'n'}

    if promotion_piece in promotion_dict:
        return chess.Move(from_square, to_square, promotion=chess.Piece.from_symbol(promotion_dict[promotion_piece]))
    return chess.Move(from_square, to_square)

def play_game_with_mcts(network, mcts_iterations=20, tau = 1.0):
    """
    Simula una partida utilizando MCTS.
    """
    board = chess.Board()
    states_with_policies = []
    result = None

    while not board.is_game_over():
        if board.can_claim_fifty_moves() or board.can_claim_threefold_repetition() or board.is_insufficient_material() or board.is_stalemate() or board.is_checkmate():
            result = "1/2-1/2"
            break  # Termina si se alcanzan tablas

        if result is not None and result != "1/2-1/2":
          result = board.result()

        root = Node(board)
        mcts(root, network, mcts_iterations)

        # Ajusta tau en función del progreso del juego
        tau = 1.0 if board.fullmove_number < 10 else 0.0
        move = select_move_with_temperature(root, tau=tau)

        policy_probs = np.zeros(4672)
        for move_key, child_node in root.children.items():
            move_index = move_to_index(move_key)
            policy_probs[move_index] = child_node.visits
        policy_probs /= policy_probs.sum()

        states_with_policies.append((board.copy(), policy_probs))
        board.push(move)

        if result is None:
          result = board.result()

    return states_with_policies, result


def calculate_returns(states_with_policies, result):
    """
    Calcula la puntuación a partir de los resultados del juego.
    """
    returns = []
    result_value = {"1-0": 1, "0-1": -1, "1/2-1/2": 0}.get(result, 0)

    # Añadir factor de perspectiva del jugador
    current_player = 1  # 1 para blancas, -1 para negras
    gamma = 0.99
    n = len(states_with_policies)
    for i in range(n):
        discounted_return = 0
        for k in range(i, n):
            discounted_return += (gamma ** (k - i)) * result_value * current_player
        returns.append(discounted_return)
        current_player *= -1  # Alternar entre jugadores

    return returns

def generate_training_data_with_temperature(num_games, network, mcts_iterations=20, tau=1.0):
    """"
    Genera datos de entrenamiento utilizando MCTS con temperatura para regular la exploración/explotación.
    """
    games_data = []
    moves_per_game = []

    for game_num in range(num_games):
        states, result = play_game_with_mcts(network, mcts_iterations, tau)
        returns = calculate_returns(states, result)
        moves_per_game.append(len(states))

        print(f"Game {game_num + 1}: {len(states)} moves, Result: {result}")

        for i, (state, policy) in enumerate(states):
            board_tensor = board_to_tensor(state)
            games_data.append((board_tensor, policy, returns[i]))

    print(f"\nAverage moves per game: {sum(moves_per_game)/len(moves_per_game):.1f}")
    return games_data

In [ ]:
def save_training_data_to_drive(training_data, output_dir, file_counter):
    """
    Guarda los datos de entrenamiento en Google Drive.
    """
    os.makedirs(output_dir, exist_ok=True)

    if training_data:
        boards, policies, values = zip(*training_data)
        boards = np.array(boards)
        policies = np.array(policies)
        values = np.array(values)
        file_path = os.path.join(output_dir, f'training_batch_{file_counter}.npz')
        np.savez_compressed(file_path, boards=boards, policies=policies, values=values)
        print(f"Datos de entrenamiento guardados en Google Drive en el archivo: {file_path}!")
    else:
        print("No hay datos de entrenamiento para guardar.")


def load_training_data_from_drive(data_dir):
    """
    Carga los datos de entrenamiento desde Google Drive.
    """
    boards_list = []
    policies_list = []
    values_list = []

    for filename in os.listdir(data_dir):
        if filename.startswith("training_batch_") and filename.endswith(".npz"):
            file_number = filename.split("_")[2].split(".")[0]
            try:
               file_number = int(file_number)
            except ValueError:
                continue

            data = np.load(os.path.join(data_dir, filename))
            boards_list.append((file_number,data["boards"]))
            policies_list.append((file_number,data["policies"]))
            values_list.append((file_number,data["values"]))

    boards_list.sort(key=lambda x: x[0])
    policies_list.sort(key=lambda x: x[0])
    values_list.sort(key=lambda x: x[0])

    boards = np.concatenate([x[1] for x in boards_list], axis=0) if boards_list else np.array([])
    policies = np.concatenate([x[1] for x in policies_list], axis=0) if policies_list else np.array([])
    values = np.concatenate([x[1] for x in values_list], axis=0) if values_list else np.array([])


    print(f'Boards data shape: {boards.shape}')
    print(f'Policies data shape: {policies.shape}')
    print(f'Values data shape: {values.shape}')
    return boards, policies, values

def save_model(model, model_path):
    """
    Guarda el modelo en el sistema de archivos.
    """
    model.save(model_path)
    print(f"Model saved at: {model_path}")

def load_model(model_path):
    """
    Carga el modelo desde el sistema de archivos.
    """
    if os.path.exists(model_path):
       model = models.load_model(model_path)
       print(f"Model loaded from: {model_path}")
       return model
    print(f"No model found at: {model_path}, new model created")
    return create_chess_network()

def data_generation_loop(num_games_per_batch, mcts_iterations, output_dir, file_counter, chess_network, tau):
    """"
    Bucle de generación de datos de entrenamiento.
    """
    training_data = generate_training_data_with_temperature(num_games_per_batch, chess_network, mcts_iterations, tau)
    print(f"Se han generado {len(training_data)} registros de entrenamiento.")

    save_training_data_to_drive(training_data, output_dir, file_counter)


def save_day_counter(day, counter_file):
    """
    Guarda el día actual en el archivo de conteo.
    """
    with open(counter_file, "w") as f:
        f.write(str(day))

def load_day_counter(counter_file):
    """
    Carga el día actual desde el archivo de conteo.
    """
    if os.path.exists(counter_file):
        with open(counter_file, "r") as f:
            return int(f.read())
    return 0


def main_training_loop(output_dir, epochs, batch_size, mcts_iterations):
    """
    Bucle principal de entrenamiento.
    """
    model_path = os.path.join(output_dir, "chess_model")
    chess_network = load_model(model_path)

    counter_file = os.path.join(output_dir, "day_counter.txt")
    day = load_day_counter(counter_file)

    if day == 0:
       print("Initial training data generation and training")
       num_games = 50
       file_counter = 0
       tau = 1.0
       data_generation_loop(num_games, mcts_iterations, output_dir, file_counter, chess_network, tau)
       try:
            boards, policies, values = load_training_data_from_drive(output_dir)

            if len(boards) == 0:
                print("No hay datos de entrenamiento disponibles")
                return None

            history = train_network(chess_network, boards, policies, values, epochs, batch_size, output_dir)
            print(f"Initial training completed")
            save_model(chess_network, model_path)

       except (FileNotFoundError, ValueError) as e:
          print(f"Error during training: {str(e)}")
          return None

    for current_day in range(day, 5):
        print(f"Starting training day: {current_day+1}")
        file_counter = current_day + 1
        num_games = 100
        tau = max(0.1, 1.0 - (current_day / 2.0))
        data_generation_loop(num_games, mcts_iterations, output_dir, file_counter, chess_network, tau)

        try:
            boards, policies, values = load_training_data_from_drive(output_dir)

            if len(boards) == 0:
                print("No hay datos de entrenamiento disponibles")
                continue

            history = train_network(chess_network, boards, policies, values, epochs, batch_size, output_dir)
            print(f"Training completed for day: {current_day+1}")
            save_model(chess_network, model_path)


        except (FileNotFoundError, ValueError) as e:
            print(f"Error during training on day {current_day+1}: {str(e)}")
        save_day_counter(current_day + 1, counter_file)

    return chess_network

In [ ]:
output_dir = '/content/drive/MyDrive/chess_bot_data'
epochs = 25
batch_size = 32
mcts_iterations = 15

# Run the training loop
trained_network = main_training_loop(output_dir, epochs, batch_size, mcts_iterations)

print("Entrenamiento completado")

Se han truncado las últimas 5000 líneas del flujo de salida.
1/1 [==============================] - 0s 25ms/step
Game 38: 54 moves, Result: 1/2-1/2
1/1 [==============================] - 0s 25ms/step
Game 39: 42 moves, Result: 1/2-1/2
1/1 [==============================] - 0s 26ms/step
Game 40: 36 moves, Result: 1/2-1/2
1/1 [==============================] - 0s 24ms/step
Game 41: 36 moves, Result: 1/2-1/2
1/1 [==============================] - 0s 26ms/step
Game 42: 36 moves, Result: 1/2-1/2
1/1 [==============================] - 0s 26ms/step
Game 43: 36 moves, Result: 1/2-1/2
1/1 [==============================] - 0s 26ms/step
Game 44: 83 moves, Result: 1/2-1/2
1/1 [==============================] - 0s 24ms/step
Game 45: 36 moves, Result: 1/2-1/2
1/1 [==============================] - 0s 25ms/step
